In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt


%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set_theme(style="whitegrid")

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os

In [7]:

dataset = "~/Desktop/uncleaned_accidents.csv"#be aware to locate the file

df = pd.read_csv(dataset)
dfo = pd.read_csv(dataset)

In [8]:
df = dfo#in case lose and you do not need to run it for long time

In [9]:
#filter the data to only country san diego
df = df[(df['County'] == 'San Diego')]
df.shape

(104165, 46)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104165 entries, 42875 to 7728390
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     104165 non-null  object 
 1   Source                 104165 non-null  object 
 2   Severity               104165 non-null  int64  
 3   Start_Time             104165 non-null  object 
 4   End_Time               104165 non-null  object 
 5   Start_Lat              104165 non-null  float64
 6   Start_Lng              104165 non-null  float64
 7   End_Lat                67640 non-null   float64
 8   End_Lng                67640 non-null   float64
 9   Distance(mi)           104165 non-null  float64
 10  Description            104165 non-null  object 
 11  Street                 103860 non-null  object 
 12  City                   104165 non-null  object 
 13  County                 104165 non-null  object 
 14  State                  104165 non-nu

In [11]:
#to check the missing part in percentage and drop the colimn with much mssing part.
missing_percentages = df.isnull().mean() * 100
threshold = 25
columns_to_drop = missing_percentages[missing_percentages > threshold].index
dropped_columns = list(columns_to_drop)
df = df.drop(columns_to_drop, axis=1)

# Drop rows with null values
df = df.dropna()

print(df.shape, dfo.shape)
print(dropped_columns)



(94450, 42) (7728394, 46)
['End_Lat', 'End_Lng', 'Wind_Chill(F)', 'Precipitation(in)']


In [12]:
# Iterate over each column
for column_name in df.columns:
    unique_values = df[column_name].unique()
    if len(unique_values) > 5:
        unique_values = unique_values[:5]
    
    unique_values_str = '|| '.join(str(value) for value in unique_values)
    
    # Print the column name followed by unique values
    print(f"{column_name}: {unique_values_str}")

ID: A-42876|| A-42887|| A-42888|| A-42889|| A-42896
Source: Source2|| Source3|| Source1
Severity: 2|| 3|| 1|| 4
Start_Time: 2016-06-21 10:39:52|| 2016-06-21 10:57:39|| 2016-06-21 10:50:48|| 2016-06-21 10:50:13|| 2016-06-21 11:12:25
End_Time: 2016-06-21 11:24:52|| 2016-06-21 11:27:39|| 2016-06-21 11:20:48|| 2016-06-21 11:20:13|| 2016-06-21 11:57:25
Start_Lat: 33.170689|| 32.765957|| 32.932255|| 33.093815|| 32.702853999999995
Start_Lng: -117.20784|| -117.128319|| -117.112679|| -117.084366|| -117.01297
Distance(mi): 0.0|| 0.01|| 0.46|| 1.87|| 0.77
Description: Accident on Palmyra Dr at Primrose Ave.|| Accident on I-805 Northbound at Exit 17A 32nd St.|| HOV lane blocked due to accident on I-15 Northbound at Exit 17 Scripps Poway Pkwy.|| Right hand shoulder blocked due to accident on I-15 Southbound at Exit 29 Gamble Ln.|| Accident on Elkelton Blvd at Paradise Valley Rd.
Street: Primrose Ave|| I-805 N|| Escondido Fwy S|| Elkelton Blvd|| Calle Fortunada
City: Vista|| San Diego|| Escondido|| 

In [13]:
columns_to_drop = ['Source', 'Start_Lat', 'Start_Lng', 'Turning_Loop','Airport_Code','Country','Zipcode','Timezone','Civil_Twilight','Nautical_Twilight','Astronomical_Twilight','Weather_Timestamp','State','ID','County']
# Drop the unnecessary columns from the DataFrame
df = df.drop(columns_to_drop, axis=1)

In [14]:
#to ensure there will no fraction after seconds.
df['Start_Time'] = df['Start_Time'].str[:19]
df['End_Time'] = df['End_Time'].str[:19]
#convert the time to datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])

In [15]:
#create Duration in min and day
df['Duration(min)'] = (round((df['End_Time'] - df['Start_Time']) / np.timedelta64(1, 'm'), 2)).astype(float)
df['Duration(day)'] = (df['End_Time'] - df['Start_Time']).dt.days
df['Duration(day)'].unique()

array([ 0,  1, 14,  3])

In [16]:
#those are some outliers will be droped
rows = df.loc[df['Duration(day)'] > 1]
df = df.drop(df[df['Duration(day)'] > 1].index)
rows

,Severity,Start_Time,End_Time,Distance(mi),Description,Street,City,Temperature(F),Humidity(%),Pressure(in),...,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Sunrise_Sunset,Duration(min),Duration(day)
6026568,4,2021-08-31 14:26:42,2021-09-14 23:59:00,0.174,San Diego Fwy S - I-5 S is closed from Birming...,I-5 S,Cardiff By the Sea,68.0,84.0,29.44,...,False,False,False,False,False,False,False,Day,20732.30,14
6437919,2,2021-01-21 13:18:00,2021-01-24 16:29:54,0.180,OLD HWY 395 JWO 15. 1039 NK ETA 30 MIN,Old Highway 395,Escondido,63.0,65.0,29.67,...,False,False,False,False,False,False,False,Day,4511.90,3
7683781,2,2019-06-03 18:03:12,2019-06-07 17:59:38,0.218,At CA-163/6th Ave/10th Ave/Exit 16 - Accident....,3rd Ave,San Diego,65.0,75.0,29.89,...,False,False,False,False,False,False,False,Day,5756.43,3


In [17]:
## give a new column about the weekday
df['Weekday'] = df['Start_Time'].apply(lambda x: pd.to_datetime(x).strftime('%A'))
df['Weekday'].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday',
       'Monday'], dtype=object)

In [18]:
#nomalize the wind direction
df['Wind_Direction'] = df['Wind_Direction'].str.lower().str.strip()
direction_mapping = {
    'w': 'w',
    'west': 'w',
    'e': 'e',
    'east': 'e',
    'north': 'n',
    'n':'n',
    'south': 's',
    's':'s',
    'calm':'calm',
    'wnw': 'wnw',
    'wsw': 'wsw',
    'sw': 'sw',
    'nw': 'nw',
    'ssw': 'ssw',
    'var': 'variable',
    'variable':'variable',
    'nnw': 'nnw',
    'sse': 'sse',
    'ene': 'ene',
    'ne': 'ne',
    'se': 'se',
    'ese': 'ese',
    'nne': 'nne',
}
df['Wind_Direction'] = df['Wind_Direction'].map(direction_mapping)
value_counts1 = df['Wind_Direction'].value_counts()
print(value_counts1)

Wind_Direction
calm        14683
w           14209
wnw         10279
wsw          8912
sw           6620
nw           6220
variable     5516
ssw          5116
s            4958
e            3212
nnw          2717
sse          2155
ene          2082
n            2079
ne           1712
se           1510
ese          1401
nne          1066
Name: count, dtype: int64


In [19]:
##encoding day and night for sunrise and sunset
mapping_1 = {'Day':1, 'Night': 0}
df['Sunrise_Sunset'] = (df['Sunrise_Sunset'].replace(mapping_1))


In [20]:
#move the position of the column
weekday_col1 = df['Weekday']
weekday_col2 = df['Duration(min)']
weekday_col3 = df['Duration(day)']
df = df.drop(columns='Weekday') 
df = df.drop(columns='Duration(min)') 
df = df.drop(columns='Duration(day)') 
df.insert(df.columns.get_loc('Start_Time'), 'Weekday', weekday_col1) 
df.insert(df.columns.get_loc('Start_Time'), 'Duration(min)', weekday_col2) 
df.insert(df.columns.get_loc('Start_Time'), 'Duration(day)', weekday_col3) 


In [21]:
# Iterate over each column
for column_name in df.columns:
    unique_values = df[column_name].unique()
    if len(unique_values) > 5:
        unique_values = unique_values[:5]
    
    unique_values_str = '|| '.join(str(value) for value in unique_values)
    
    # Print the column name followed by unique values
    print(f"{column_name}: {unique_values_str}")

Severity: 2|| 3|| 1|| 4
Weekday: Tuesday|| Wednesday|| Thursday|| Friday|| Saturday
Duration(min): 45.0|| 30.0|| 60.0|| 40.43|| 42.63
Duration(day): 0|| 1
Start_Time: 2016-06-21 10:39:52|| 2016-06-21 10:57:39|| 2016-06-21 10:50:48|| 2016-06-21 10:50:13|| 2016-06-21 11:12:25
End_Time: 2016-06-21 11:24:52|| 2016-06-21 11:27:39|| 2016-06-21 11:20:48|| 2016-06-21 11:20:13|| 2016-06-21 11:57:25
Distance(mi): 0.0|| 0.01|| 0.46|| 1.87|| 0.77
Description: Accident on Palmyra Dr at Primrose Ave.|| Accident on I-805 Northbound at Exit 17A 32nd St.|| HOV lane blocked due to accident on I-15 Northbound at Exit 17 Scripps Poway Pkwy.|| Right hand shoulder blocked due to accident on I-15 Southbound at Exit 29 Gamble Ln.|| Accident on Elkelton Blvd at Paradise Valley Rd.
Street: Primrose Ave|| I-805 N|| Escondido Fwy S|| Elkelton Blvd|| Calle Fortunada
City: Vista|| San Diego|| Escondido|| Spring Valley|| Santee
Temperature(F): 69.1|| 73.0|| 75.2|| 71.1|| 70.0
Humidity(%): 84.0|| 75.0|| 66.0|| 61.0||

In [22]:
df['Weather_Condition'].unique()

array(['Partly Cloudy', 'Scattered Clouds', 'Mostly Cloudy', 'Clear',
       'Overcast', 'Haze', 'Fog', 'Mist', 'Light Rain', 'Rain',
       'Heavy Rain', 'Thunderstorms and Rain', 'Hail',
       'Light Thunderstorms and Rain', 'Drizzle', 'Light Drizzle', 'Fair',
       'Cloudy', 'Fair / Windy', 'Heavy Rain / Windy',
       'Light Rain / Windy', 'T-Storm', 'Thunder',
       'Thunder in the Vicinity', 'Rain / Windy', 'Cloudy / Windy',
       'Mostly Cloudy / Windy', 'Partly Cloudy / Windy', 'Shallow Fog',
       'Smoke', 'Patches of Fog', 'Blowing Dust',
       'Light Rain with Thunder', 'Heavy T-Storm',
       'Heavy Thunderstorms and Rain'], dtype=object)

In [23]:
# Standardize weather condition labels
df['Weather_Condition'] = df['Weather_Condition'].str.lower()
weather_mapping = {
    'partly cloudy': '',
    'scattered clouds': 'cloudy',
    'mostly cloudy': 'cloudy',
    'clear': 'clear',
    'overcast': 'cloudy',
    'haze': 'mist',
    'fog': 'mist',
    'mist': 'mist',
    'light rain': 'rain',
    'rain': 'rain',
    'heavy rain': 'rain',
    'thunderstorms and rain': 'rain',
    'hail': 'rain',
    'light thunderstorms and rain': 'rain',
    'drizzle': 'rain',
    'light drizzle': 'rain',
    'fair': 'clear',
    'cloudy': 'cloudy',
    'fair / windy': 'clear',
    'heavy rain / windy': 'rain',
    'light rain / windy': 'rain',
    't-storm': 'rain',
    'thunder': 'rain',
    'thunder in the vicinity': 'rain',
    'rain / windy': 'rain',
    'cloudy / windy': 'cloudy',
    'mostly cloudy / windy': 'cloudy',
    'partly cloudy / windy': 'cloudy',
    'shallow fog': 'mist',
    'smoke': 'mist',
    'patches of fog': 'mist',
    'blowing dust': 'mist',
    'light rain with thunder': 'rain',
    'heavy t-storm': 'rain',
    'heavy thunderstorms and rain': 'rain',

}

df['Weather_Condition'] = df['Weather_Condition'].map(weather_mapping)
# Encode weather condition as a categorical variable
df['Weather_Condition'] = df['Weather_Condition'].astype('category')
df = df.dropna()
print(df['Weather_Condition'].unique())


['', 'cloudy', 'clear', 'mist', 'rain']
Categories (5, object): ['', 'clear', 'cloudy', 'mist', 'rain']


In [24]:
df['Weather_Condition']

42875            
42886      cloudy
42887      cloudy
42888            
42895       clear
            ...  
7727956     clear
7727960     clear
7728374     clear
7728377     clear
7728390     clear
Name: Weather_Condition, Length: 94447, dtype: category
Categories (5, object): ['', 'clear', 'cloudy', 'mist', 'rain']

In [25]:
columns_to_convert = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal']

df[columns_to_convert] = df[columns_to_convert].astype(int)


In [26]:
#df.to_csv("~/Desktop/cogs108/109cleaned.csv", index=False)


In [27]:
df.shape

(94447, 30)